# Web Scraping and Introductory Data Analysis
// purpose of assintment


// summary of what we're going to do

In [1]:
%pip install -r ./../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [23]:
NUMBER_BLOCKS = 10
ETHERESCAN_URL = "https://etherscan.io/txs"

In [18]:
class Scrip_Etheruim:
    url = ETHERESCAN_URL

    def __init__(self):
        self.driver = webdriver.Chrome()
        self.df = pd.DataFrame(columns=['tnx_hash', 'method', 'block', 'date', 'from', 'to', 'value', 'tnx_fee'])

    def __del__(self):
        self.driver.quit()

    def _get_text(self, element):
        return element.find('a').get('href').split('/')[-1] 

    def _get_data(self, element):
        tnx_hash = element[1].text.strip()
        method = element[2].text.strip()
        block = element[3].text.strip()
        date = element[4].text.strip()
        _from = self._get_text(element[7])
        to = self._get_text(element[9])
        value = element[10].text.strip()
        tnx_fee = element[11].text.strip()
        data = [tnx_hash, method, block, date, _from, to, value, tnx_fee]
        return pd.Series(data, index=['tnx_hash', 'method', 'block', 'date', 'from', 'to', 'value', 'tnx_fee']), block


    def _extract_data_from_html(self, html_content):
        soup = BeautifulSoup(html_content, "html.parser")
        rows = soup.find_all("tr")
        block_number = 0
        for row in rows:
            cells = row.find_all("td")
            series, block = self._get_data(cells)
            block_number = max(block_number, block)
            self.df = pd.concat([self.df, pd.DataFrame([series])], ignore_index=True)

        return block_number

            
    def _extract_data_from_url(self):
        table_body = self.driver.find_element(By.CSS_SELECTOR, "tbody.align-middle.text-nowrap")
        html_content = table_body.get_attribute("outerHTML")
        block_number = self._extract_data_from_html(html_content)
        time.sleep(0.5)
        return block_number
    
    def click_next_button(self):
        """Clicks the "Next" button on the provided HTML structure.

        Args:
            driver (webdriver.Chrome): The Selenium WebDriver instance.

        Raises:
            Exception: If the button cannot be found or clicked.
        """

        try:
            # Use CSS selector for flexibility and maintainability
            next_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "a[aria-label='Next']"))
            )
            next_button.click()
        except Exception as e:
            print(f"Error clicking the 'Next' button: {e}")
            raise

    def extract_data(self):
        self.driver.get(self.url)
        block_number = self._extract_data_from_url()
        new_block_number = block_number

        while(block_number-new_block_number < NUMBER_BLOCKS):
            try:
                element = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.ID, "myDynamicElement"))
                )
                element.click()
                self._extract_data_from_url()
            except:
                print("Timeout")
      





In [21]:
def click_next_button(driver):
    """Clicks the "Next" button on the provided HTML structure.

    Args:
        driver (webdriver.Chrome): The Selenium WebDriver instance.

    Raises:
        Exception: If the button cannot be found or clicked.
    """

    try:
        # Use CSS selector for flexibility and maintainability
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a[aria-label='Next']"))
        )
        next_button.click()
    except Exception as e:
        print(f"Error clicking the 'Next' button: {e}")
        raise



    

In [29]:
driver = webdriver.Chrome()
driver.get(ETHERESCAN_URL)

next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a[aria-label='Next']"))
)
# click_next_button(driver)
time.sleep(5)
driver.quit()


MaxRetryError: HTTPConnectionPool(host='localhost', port=41285): Max retries exceeded with url: /session/5903c4ffc5fe8b95fd135e74a5896c90/element/f.9DF270808D123FC2A127AD15A2C82DF7.d.E7E3A6C089F125A5C608C596FD7D11B6.e.87/text (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7ff2588ca810>: Failed to establish a new connection: [Errno 111] Connection refused'))